<a href="https://colab.research.google.com/github/kumarsonam/Colab/blob/main/hello_world_cpp_cuda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvcc --version


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [2]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git


  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-vtqhybd4
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-vtqhybd4
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0d2ab99cccbbc682722e708515fe9c4cfc50185a
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4716 sha256=ea8bdd036a976740380b4c7bcd6b6f552faafaa0c6f718605cc8011cf90b296b
  Stored in directory: /tmp/pip-ephem-wheel-cache-tie_8jy4/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [3]:
%load_ext nvcc_plugin


created output directory at /content/src
Out bin /content/result.out


In [4]:
%%cu
#include <iostream>
    int
    main()
{
    std::cout << "Welcome To GeeksforGeeks\n";
    return 0;
}

Welcome To GeeksforGeeks



In [14]:
!apt-get install -y libthrust-dev


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libcub-dev
Suggested packages:
  nvidia-cuda-toolkit
The following NEW packages will be installed:
  libcub-dev libthrust-dev
0 upgraded, 2 newly installed, 0 to remove and 24 not upgraded.
Need to get 639 kB of archives.
After this operation, 8,233 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libcub-dev all 1.15.0-3 [217 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/multiverse amd64 libthrust-dev all 1.15.0-1 [423 kB]
Fetched 639 kB in 1s (877 kB/s)
Selecting previously unselected package libcub-dev.
(Reading database ... 121654 files and directories currently installed.)
Preparing to unpack .../libcub-dev_1.15.0-3_all.deb ...
Unpacking libcub-dev (1.15.0-3) ...
Selecting previously unselected package libthrust-dev.
Preparing to unpack .../libthrust-dev_1.15.0-1_all.deb ...
Un

In [18]:
%%cu
#include <stdio.h>

__global__ void helloCUDA() {
    printf("Hello, CUDA World using ChatGPT!\n");
}

int main() {
    // Launch the helloCUDA kernel on the GPU
    helloCUDA<<<1, 1>>>();

    // Wait for GPU to finish before exiting
    cudaDeviceSynchronize();

    return 0;
}

Hello, CUDA World using ChatGPT!



In [19]:
%%cu
#include <stdio.h>

__global__ void findMax(int* input, int* output, int size) {
    int tid = threadIdx.x;

    // Allocate shared memory
    extern __shared__ int shared_mem[];

    // Initialize shared memory
    if (tid < size) {
        shared_mem[tid] = input[tid];
    } else {
        shared_mem[tid] = 0;  // Initialize to a reasonable default value
    }

    // Synchronize threads
    __syncthreads();

    // Reduction to find the maximum element
    for (int stride = 1; stride < blockDim.x; stride *= 2) {
        int index = 2 * stride * tid;
        if (index < blockDim.x) {
            int temp = shared_mem[index + stride];
            if (temp > shared_mem[index]) {
                shared_mem[index] = temp;
            }
        }
        __syncthreads();
    }

    // Store the result in the output array
    if (tid == 0) {
        *output = shared_mem[0];
    }
}

int main() {
    // Vector size
    int size = 10;

    // Host vector
    int host_vector[] = {4, 2, 8, 5, 1, 7, 9, 3, 6, 10};

    // Device vectors
    int* dev_input;
    int* dev_output;

    // Allocate memory on the GPU
    cudaMalloc((void**)&dev_input, size * sizeof(int));
    cudaMalloc((void**)&dev_output, sizeof(int));

    // Copy data from host to device
    cudaMemcpy(dev_input, host_vector, size * sizeof(int), cudaMemcpyHostToDevice);

    // Launch the findMax kernel on the GPU
    findMax<<<1, size, size * sizeof(int)>>>(dev_input, dev_output, size);

    // Wait for GPU to finish before retrieving results
    cudaDeviceSynchronize();

    // Copy the result back to the host
    int result;
    cudaMemcpy(&result, dev_output, sizeof(int), cudaMemcpyDeviceToHost);

    // Print the result
    printf("Maximum element: %d\n", result);

    // Free allocated memory on the GPU
    cudaFree(dev_input);
    cudaFree(dev_output);

    return 0;
}

Maximum element: 10

